In [15]:
from sklearn import cross_validation, linear_model, metrics
from math import sqrt
from statsmodels.stats.weightstats import _tconfint_generic



import scipy
import numpy as np
import pandas as pd

In [2]:
testData = pd.DataFrame()
testData = testData.from_csv("water.txt", sep="	", index_col=1)

In [3]:
testData

,location,mortality,hardness
town,,,
Bath,South,1247,105
Birkenhead,North,1668,17
Birmingham,South,1466,5
Blackburn,North,1800,14
Blackpool,North,1609,18
Bolton,North,1558,10
Bootle,North,1807,15
Bournemouth,South,1299,78
Bradford,North,1637,10


In [4]:
mortality = testData.as_matrix(["mortality"])
meanMortality = np.mean(mortality)
mortalityStd = mortality.std(ddof=1)/sqrt(len(mortality))
_tconfint_generic(meanMortality, mortalityStd,len(mortality) - 1, 0.05, 'two-sided')

(1476.0833413552848, 1572.2117406119285)

In [5]:
filteredMortality = []
for row in testData.as_matrix(["hardness", "location"]):
    if (row[1] == 'South'):
        filteredMortality.append(row[0])

npfilteredMortality = np.array(filteredMortality)

meanFilteredMortality = np.mean(npfilteredMortality)
meanFilteredMortalityStd = npfilteredMortality.std(ddof=1)/sqrt(len(npfilteredMortality))
_tconfint_generic(meanFilteredMortality, meanFilteredMortalityStd,len(npfilteredMortality) - 1, 0.05, 'two-sided')
#ЮГ (1320.1517462936238, 1433.463638321761)
#Север (1586.5605251961385, 1680.6394748038613)

#Жёсткость север (21.422487285724259, 39.377512714275738)
#Жёсткость Юг (53.467198692036106, 86.071262846425441)

(53.467198692036106, 86.071262846425441)

In [6]:
npfilteredMortality = np.zeros(11037)
for value in range(104):
    npfilteredMortality[value] = 1
print npfilteredMortality

[ 1.  1.  1. ...,  0.  0.  0.]


In [7]:
#Задача про майа
from statsmodels.stats.proportion import proportion_confint
normal_interval = proportion_confint(sum(npfilteredMortality), len(npfilteredMortality), method = 'normal')
normal_interval

(0.0076204226382885678, 0.011225278186210842)

In [8]:
from statsmodels.stats.proportion import samplesize_confint_proportion
n_samples = int(np.ceil(samplesize_confint_proportion(npfilteredMortality.mean(), 0.01)))
n_samples

359

In [9]:
from scipy.stats import norm

In [10]:
norm.ppf(0.9985, 0, 1)

2.9677379253417944

In [11]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [17]:
aspirin = np.zeros(11037)
for value in range(104):
    aspirin[value] = 1

withoutAspirin = np.zeros(11034)
for value in range(189):
    withoutAspirin[value] = 1

print proportions_confint_diff_ind(withoutAspirin, aspirin)

(0.0046877506750494392, 0.010724297276960124)


In [33]:
p_aspirin = (104. / 11037.)
shans_aspirin = 104. / (11037. - 104.)

p_withotAspirin = (189. / 11034.)
shans_withotAspirin = 189 / (11034. - 189)

In [40]:
print    shans_withotAspirin / shans_aspirin

1.83205394191


In [37]:
print p_withotAspirin - p_aspirin

0.007706023976
